In [ ]:
import pandas as pd
from cassis import load_typesystem, load_cas_from_xmi
import os
import zipfile

In [ ]:
# This is the 'annotation 'subfolder of the unzipped export file
anndir = "data/annotated-papers"

# The folder contains subfolders for all documents in inception 
# We loop over these subfolders
for docdir in os.listdir(anndir):

    fulldocdir = os.path.join(anndir,docdir)
    
    if zipfile.is_zipfile(fulldocdir):
        print(fulldocdir)
        z = zipfile.ZipFile(fulldocdir)
        l = z.namelist()
        print(l)
        ts = load_typesystem(z.open('TypeSystem.xml').read().decode())
        xmi = l[0] if l[0] != 'TypeSystem.xml' else l[1]   # get name of other file in zipfile
        annotator = xmi.split('.')[0]                      # annotator name derived from filename
        cas = load_cas_from_xmi(z.open(xmi).read().decode(), typesystem=ts)

        with open("data/annotated-papers/output/output_" + docdir, 'w') as f:
            for segment in cas.select('custom.SentenceLabel'):
                f.write(f"{segment.get_covered_text()}\t{segment.label}\n")

In [ ]:
list_dfs = []

for docdir in os.listdir(anndir):
    fulldocdir = os.path.join(anndir, docdir)
    if zipfile.is_zipfile(fulldocdir):
        # Create empty lists to store data
        sentences = []
        classes = []

        # Read the text file line by line
        with open("data/annotated-papers/output/output_" + docdir, 'r', errors='replace') as file:
            lines = file.readlines()

        # Process each line and extract sentences and classes
        for line in lines:
            line = line.strip()
            if line:
                parts = line.split('\t')
                if len(parts) == 2:
                    sentence, class_label = parts
                    sentences.append(sentence)
                    classes.append(class_label)

        # Create a pandas DataFrame
        df = pd.DataFrame({"title_cod": docdir[15:-4], "text": sentences, "labels": classes})

        list_dfs.append(df)

In [4]:
df = pd.concat(list_dfs)

In [ ]:
df.to_csv("results/df_labelled_sentences.csv")